In [5]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math

In [51]:
def imShow(image):
    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    #plt.rcParams['figure.figsize'] = [10, 5]
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)


def image_processing_front_voter(img):
    adjusted = adjust_gamma(img, gamma=1.7)
    # adjusted = adjust_gamma(adjusted, gamma=1.3)#1.3
    gray = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)
    res, th1 = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    res, th2 = cv2.threshold(th1, 120, 255, cv2.THRESH_BINARY)
    th3 = cv2.adaptiveThreshold(th2, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 61, 10)
    adjusted1 = adjust_gamma(th3, gamma=0.9)                                      
    return adjusted1


In [115]:
def skew_correction_voter_front(img):
    image = cv2.imread(img)
    print(image.shape)
    image = cv2.resize(image, (1000, 1500))
    imShow(image)
    gray = image_processing_front_voter(image)
    imShow(gray)
    (thresh, image_binary) = cv2.threshold(gray,0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    imShow(image_binary)
    # function call
    from pythonRLSA import rlsa
    image_rlsa_horizontal = rlsa.rlsa(image_binary, True, False,40)
    (thresh, image_rlsa_horizontal_inv) = cv2.threshold(image_rlsa_horizontal,0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    imShow(image_rlsa_horizontal_inv)
    output = cv2.connectedComponentsWithStats(image_rlsa_horizontal_inv, 4, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output

    count=0
    sum_angle=0
    positive_skew=[]
    negative_skew=[]
    filtered_numlabels=[]
    # loop over the number of unique connected component labels
    for i in range(0,numLabels):
        # if this is the first component then we examine the
        # *background* (typically we would just ignore this
        # component in our loop)
        if i == 0:
            text = "examining component {}/{} (background)".format(
                i + 1, numLabels)
        # otherwise, we are examining an actual connected component
        else:
            text = "examining component {}/{}".format( i + 1, numLabels)
            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            area = stats[i, cv2.CC_STAT_AREA]
            (cX, cY) = centroids[i]
            componentMask = (labels == i).astype("uint8") * 255
            rectangle_area=w*h
            y_connected=h-((rectangle_area-area)/w)
            x_connected=area/y_connected
            alpha=x_connected/y_connected
            
            xm,ym=(x,y+(y_connected/2))
            a=(cY-ym)/(cX-xm)

            radian=math.atan(a)

            angle=radian*(180/math.pi)

            if w>=300 and alpha>=3:
                filtered_numlabels.append(i)
    if len(filtered_numlabels)>4:
        final_numlabels=[filtered_numlabels[0],filtered_numlabels[1],filtered_numlabels[-2],filtered_numlabels[-1]]
    else:
        final_numlabels=filtered_numlabels
    print(final_numlabels)   
    for i in final_numlabels:
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        (cX, cY) = centroids[i]
        componentMask = (labels == i).astype("uint8") * 255
        rectangle_area=w*h
        y_connected=h-((rectangle_area-area)/w)
        x_connected=area/y_connected
        alpha=x_connected/y_connected

        xm,ym=(x,y+(y_connected/2))
        a=(cY-ym)/(cX-xm)

        radian=math.atan(a)

        angle=radian*(180/math.pi)

        sum_angle+=angle
        count+=1
        output = image_rlsa_horizontal.copy()
        componentMask = (labels == i).astype("uint8") * 255

        componentMask_copy=componentMask.copy()
        cv2.rectangle(componentMask_copy, (x, y), (x + w, y + h), (255, 255, 255), 1)
        imShow(componentMask_copy)
        cv2.rectangle(output, (x, y), (x + w, y + h), (255, 255, 255), 1)
        white_list=[]
        black_list=[]
        for i in range(int(xm),int(cX)):
            y_line=int(a*(float(i)-xm)+ym)
            x_line=int(i)

            if componentMask[y_line,x_line]==255:         
                white_list.append(255)

            elif componentMask[y_line,x_line]==0:
                black_list.append(0)
            #cv2.circle(componentMask_copy, (x_line,y_line), radius=0, color=(255, 255, 255), thickness=0)
        if len(black_list)>len(white_list):
            positive_skew.append('True')
        else:
            negative_skew.append('True')
        if count>0:
            avg_skew_angle=sum_angle/count
        else:
            avg_skew_angle=sum_angle
        if len(positive_skew)>len(negative_skew):           
                rotation_angle=360-avg_skew_angle
        else:
            rotation_angle=avg_skew_angle
    print(rotation_angle)
    skew_allowable=0.5
    image=cv2.imread(img)
    if avg_skew_angle>skew_allowable:
        (h, w) = image.shape[:2]
        (cX, cY) = (w // 2, h // 2)
        # rotate our image by 45 degrees around the center of the image
        M = cv2.getRotationMatrix2D((cX, cY), rotation_angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h))
        return rotated
    else:
        return image

In [1]:
res=skew_correction_voter_front('upload_your_voter_front.jpg')
imShow(res)

NameError: name 'skew_correction_voter_front' is not defined